In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import os
import sys

sys.path.append('../model')

import config
from scenario import *
from tileset import *

%matplotlib inline

In [2]:
scenarios = []
scenarios += process_scenarios(os.path.join(config.STARCRAFT_ROOT, 'Maps'))
for directory in config.MAP_DIRECTORIES:
    scenarios += process_scenarios(directory)

jungle_scenarios = [x for x in scenarios if x.tileset == Tileset.JUNGLE]

len(jungle_scenarios)

223

In [3]:
jungle_tiles = Tileset.JUNGLE.tiles
terrain_tiles = (tile for tile in jungle_tiles if not tile.is_doodad)
doodad_tiles = (tile for tile in jungle_tiles if tile.is_doodad)

groups = dict()
tile_features = dict()

for tile in terrain_tiles:
    if tile not in tile_features:
        if tile.group_id not in groups:
            groups[tile.group_id] = len(groups)
        tile_features[tile] = (groups[tile.group_id], tile.group_offset + 1)

tile_kinds = len(groups)
for tile in doodad_tiles:
    if tile not in tile_features:
        tile_features[tile] = (tile_kinds, 0)
        tile_kinds += 1

def scenario_data(scenarios):
    for scenario in scenarios:
        for i in range(scenario.tiles.shape[0]):
            for j in range(scenario.tiles.shape[1]):
                left_tile = scenario.tiles[i, j - 1] if j != 0 else jungle_tiles[0]
                top_tile = scenario.tiles[i - 1, j] if i != 0 else jungle_tiles[0]
                tile = scenario.tiles[i, j]

                yield [*tile_features[top_tile], *tile_features[left_tile], *tile_features[tile]]

In [12]:
dataset = list(scenario_data(x for x in jungle_scenarios if x.width == 64 and x.height == 64))
dataset = np.array(dataset)
dataset.shape

(49152, 6)

In [7]:
top_tile_kind_column = tf.feature_column.categorical_column_with_identity(key='top_tile_kind', num_buckets=tile_kinds)
top_tile_variant_column = tf.feature_column.categorical_column_with_identity(key='top_tile_variant', num_buckets=17)
left_tile_kind_column = tf.feature_column.categorical_column_with_identity(key='left_tile_kind', num_buckets=tile_kinds)
left_tile_variant_column = tf.feature_column.categorical_column_with_identity(key='left_tile_variant', num_buckets=17)
tile_kind_column = tf.feature_column.categorical_column_with_identity(key='tile_kind', num_buckets=tile_kinds)
tile_variant_column = tf.feature_column.categorical_column_with_identity(key='tile_variant', num_buckets=17)

feature_columns = [top_tile_kind_column, top_tile_variant_column, left_tile_kind_column, left_tile_variant_column]

In [16]:
def input_fn():
    return {
        'top_tile_kind': dataset.T[0],
        'top_tile_variant': dataset.T[1],
        'left_tile_kind': dataset.T[2],
        'left_tile_variant': dataset.T[3],
    },
    dataset.T[4]